In [20]:
import polars as pl
import os

train_applprev_2 = pl.read_csv("Data/csv_files/train/train_applprev_2.csv")

train_applprev_2.shape

(14075487, 6)

In [2]:
most_frequent_case_id = (
    train_applprev_2.filter(pl.col("credacc_cards_status_52L").is_not_null())
    .group_by("case_id")
    .agg(pl.len().alias("count"))
    .sort("count")
    .reverse()
    .select("case_id")
    .head(1)
    .to_series()
    [0]
)

print(most_frequent_case_id)

147994


## Exploring Data Indexing with num_group1 and num_group2
### Objective
To understand how num_group1 and num_group2 index historical records associated with each case_id for accurate data flattening and analysis.

### Process
Filtered Data: Identified a frequent case_id focusing on non-null credit card statuses to understand how records are indexed.
Indexed Data Attributes: Investigated the structure revealed by num_group1 and num_group2, determining their roles in categorizing data hierarchically, e.g., num_group1 for primary categorizations and num_group2 for secondary levels within each category.
Conclusion
Understanding the indexing mechanism is crucial for data preparation, ensuring accurate feature representation for model training. The exploration provided insights into the dataset's hierarchical structure, aiding in effective data analysis.

In [3]:
filtered_df = train_applprev_2.filter(pl.col("case_id") == most_frequent_case_id)

sorted_df = filtered_df.sort(["num_group1", "num_group2"])

sorted_df.head(30)

case_id,cacccardblochreas_147M,conts_type_509L,credacc_cards_status_52L,num_group1,num_group2
i64,str,str,str,i64,i64
147994,"""a55475b1""","""PRIMARY_MOBILE…",null,0,0
147994,"""a55475b1""",null,null,0,1
147994,"""a55475b1""","""PRIMARY_MOBILE…","""CANCELLED""",1,0
147994,"""a55475b1""","""HOME_PHONE""",null,1,1
147994,"""a55475b1""","""PRIMARY_MOBILE…","""ACTIVE""",2,0
147994,"""a55475b1""","""PHONE""",null,2,1
147994,"""a55475b1""","""PRIMARY_MOBILE…","""ACTIVE""",3,0
147994,"""a55475b1""",null,null,3,1
147994,"""a55475b1""","""PRIMARY_MOBILE…","""CANCELLED""",4,0


In [4]:
train_applprev_2 = train_applprev_2.group_by(["case_id", "num_group1"]) \
                  .agg([
                      pl.col("conts_type_509L").unique().count().alias("unique_contact_types"),
                      pl.col("cacccardblochreas_147M").max().alias("first_cacccardblochreas_147M"),
                      pl.col("credacc_cards_status_52L").max().alias("first_credacc_cards_status_52L")
                  ])

In [5]:
train_applprev_2.describe()

statistic,case_id,num_group1,unique_contact_types,first_cacccardblochreas_147M,first_credacc_cards_status_52L
str,f64,f64,f64,str,str
"""count""",6.525978e6,6.525978e6,6.525978e6,"""6468134""","""316210"""
"""null_count""",0.0,0.0,0.0,"""57844""","""6209768"""
"""mean""",1.4319e6,4.194369,2.069206,null,null
"""std""",787191.344959,4.274768,0.77185,null,null
"""min""",2.0,0.0,1.0,"""P133_119_56""","""ACTIVE"""
"""25%""",958657.0,1.0,2.0,null,null
"""50%""",1.555975e6,3.0,2.0,null,null
"""75%""",1.853312e6,6.0,3.0,null,null
"""max""",2.703454e6,19.0,6.0,"""a55475b1""","""UNCONFIRMED"""


In [6]:
filtered_df_2 = train_applprev_2.filter(pl.col("case_id") == most_frequent_case_id)

sorted_df_2 = filtered_df_2.sort(["num_group1"])

sorted_df_2.head(30)

case_id,num_group1,unique_contact_types,first_cacccardblochreas_147M,first_credacc_cards_status_52L
i64,i64,u32,str,str
147994,0,2,"""a55475b1""",null
147994,1,2,"""a55475b1""","""CANCELLED"""
147994,2,2,"""a55475b1""","""ACTIVE"""
147994,3,2,"""a55475b1""","""ACTIVE"""
147994,4,2,"""a55475b1""","""CANCELLED"""
147994,5,2,"""a55475b1""","""CANCELLED"""
147994,6,3,"""a55475b1""","""CANCELLED"""
147994,7,2,"""a55475b1""","""CANCELLED"""
147994,8,1,"""a55475b1""","""ACTIVE"""


In [7]:
train_applprev_2 = train_applprev_2.with_columns(
    pl.col('first_credacc_cards_status_52L')
        .fill_null('UNKNOWN')
        .alias('status')
)

train_applprev_2 = train_applprev_2.with_columns([
    (pl.col('status') == 'ACTIVE').cast(pl.Int32).alias('is_active'),
    (pl.col('status') == 'CANCELLED').cast(pl.Int32).alias('is_cancelled')
])


train_applprev_2 = train_applprev_2.group_by('case_id').agg([
    pl.col('unique_contact_types').max().alias('max_unique_contact_type'),
    pl.col('first_cacccardblochreas_147M').n_unique().alias('n_unique_cacccardblochreas_147M'),
    pl.sum('is_cancelled').alias('total_cancelled')
])

In [17]:
train_applprev_2.head()

case_id,max_unique_contact_type,n_unique_cacccardblochreas_147M,total_cancelled
i64,u32,u32,i32
693544,2,1,0
1069,2,1,0
1547930,3,1,0
1930684,2,1,0
1434543,2,1,0


In [9]:
train_person_2 = pl.read_csv("Data/csv_files/train/train_person_2.csv")

train_person_2.shape

(1643410, 11)

In [10]:
most_frequent_case_id = (
    train_person_2.filter(pl.col("addres_district_368M").is_not_null())
    .group_by("case_id")
    .agg(pl.len().alias("count"))
    .sort("count")
    .reverse()
    .select("case_id")
    .head(2)
    .to_series()
    [1]
)

print(most_frequent_case_id)

140528


In [11]:
filtered_2_df = train_person_2.filter(pl.col("case_id") == most_frequent_case_id)

sorted_2_df = filtered_2_df.sort(["num_group1", "num_group2"])

sorted_2_df.head(50)

case_id,addres_district_368M,addres_role_871L,addres_zip_823M,conts_role_79M,empls_economicalst_849M,empls_employedfrom_796D,empls_employer_name_740M,num_group1,num_group2,relatedpersons_role_762T
i64,str,str,str,str,str,str,str,i64,i64,str
140528,"""P215_163_136""","""CONTACT""","""P85_138_173""","""P38_92_157""","""P148_57_109""",null,"""a55475b1""",0,0,null
140528,"""P215_163_136""","""PERMANENT""","""P85_138_173""","""a55475b1""","""a55475b1""",null,"""a55475b1""",0,1,null
140528,"""P215_163_136""","""CONTACT""","""P85_138_173""","""P38_92_157""","""P148_57_109""",null,"""a55475b1""",1,0,"""OTHER_RELATIVE…"
140528,"""P215_163_136""","""TEMPORARY""","""P85_138_173""","""P177_137_98""","""a55475b1""",null,"""a55475b1""",1,1,"""CHILD"""
140528,"""P215_163_136""","""PERMANENT""","""P85_138_173""","""P177_137_98""","""a55475b1""",null,"""a55475b1""",1,2,"""SIBLING"""
140528,"""P215_163_136""","""PERMANENT""","""P85_138_173""","""P38_92_157""","""a55475b1""",null,"""a55475b1""",1,3,"""OTHER_RELATIVE…"
140528,"""a55475b1""",null,"""a55475b1""","""P125_14_176""","""a55475b1""",null,"""a55475b1""",1,4,"""OTHER_RELATIVE…"
140528,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",1,5,"""OTHER_RELATIVE…"
140528,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",1,6,"""PARENT"""


In [12]:
train_person_2 = train_person_2.group_by('case_id').agg([
    pl.col('addres_district_368M').n_unique().alias('n_unique_addres_district_368M'),
    pl.col('addres_role_871L').n_unique().alias('n_unique_addres_role_871L'),
    pl.col('addres_zip_823M').n_unique().alias('n_unique_addres_zip_823M'),
    pl.col('conts_role_79M').n_unique().alias('n_unique_conts_role_79M'),
    pl.col('empls_economicalst_849M').n_unique().alias('n_unique_empls_economicalst_849M'),
    pl.col('empls_employedfrom_796D').n_unique().alias('n_unique_empls_employedfrom_796D'),
    pl.col('empls_employer_name_740M').n_unique().alias('n_unique_empls_employer_name_740M'),
    pl.col('relatedpersons_role_762T').n_unique().alias('n_unique_relatedpersons_role_762T')
])

In [18]:
train_person_2.head()

case_id,n_unique_addres_district_368M,n_unique_addres_role_871L,n_unique_addres_zip_823M,n_unique_conts_role_79M,n_unique_empls_economicalst_849M,n_unique_empls_employedfrom_796D,n_unique_empls_employer_name_740M,n_unique_relatedpersons_role_762T
i64,u32,u32,u32,u32,u32,u32,u32,u32
981528,1,1,1,1,1,1,1,1
1484877,1,1,1,1,1,1,1,1
1562617,1,1,1,1,1,1,1,1
1569292,1,1,1,1,1,1,1,1
217086,1,1,1,1,1,1,1,1


In [19]:
train_DEPTH_2 = train_applprev_2.join(
    train_person_2,
    on='case_id',
    how='inner',
)

train_DEPTH_2.head()

case_id,max_unique_contact_type,n_unique_cacccardblochreas_147M,total_cancelled,n_unique_addres_district_368M,n_unique_addres_role_871L,n_unique_addres_zip_823M,n_unique_conts_role_79M,n_unique_empls_economicalst_849M,n_unique_empls_employedfrom_796D,n_unique_empls_employer_name_740M,n_unique_relatedpersons_role_762T
i64,u32,u32,i32,u32,u32,u32,u32,u32,u32,u32,u32
1484877,2,1,0,1,1,1,1,1,1,1,1
1562617,3,1,0,1,1,1,1,1,1,1,1
1569292,2,1,0,1,1,1,1,1,1,1,1
217086,2,1,0,1,1,1,1,1,1,1,1
1749144,3,1,0,1,1,1,1,1,1,1,1


In [21]:
directory = r"C:\Users\afise\.git\CreditRiskModel\Merged_Data"
filename = "train_DEPTH_2.csv"
file_path = os.path.join(directory, filename)

if not os.path.exists(directory):
    os.makedirs(directory)

train_DEPTH_2.write_csv(file_path)